# 📘 tf.keras.callbacks 详解
## 1. 回调（Callback）的作用

- 监控训练过程：如在每个 `epoch` 结束时记录损失和准确率。

- 保存模型：训练过程中自动保存权重或最佳模型。

- 提前停止：防止过拟合，验证集指标不再提升时提前停止训练。

- 动态调整学习率：根据验证集效果自动调节学习率。

- 日志可视化：结合 `TensorBoard` 展示曲线、分布等。

---
## 2. 常用回调类
| 回调类                         | 作用                            |
| --------------------------- | ----------------------------- |
| **`ModelCheckpoint`**       | 保存模型/权重（可选择只保存验证集最佳模型）。       |
| **`EarlyStopping`**         | 当验证集指标在若干 epoch 内不再提升时提前停止训练。 |
| **`ReduceLROnPlateau`**     | 验证集指标长期不提升时，自动减小学习率。          |
| **`TensorBoard`**           | 记录训练日志，供 TensorBoard 可视化。     |
| **`CSVLogger`**             | 将每个 epoch 的日志保存到 CSV 文件。      |
| **`LearningRateScheduler`** | 自定义学习率调整策略。                   |
| **`TerminateOnNaN`**        | 如果出现 NaN 值，立即停止训练。            |
| **`ProgbarLogger`**         | 控制训练过程的进度条输出。                 |

## 3.如何使用 Callbacks？
使用 `Callbacks` 非常简单，只需在 `model.fit()` 时通过 `callbacks` 参数传入一个 `Callback` `列表`即可。

In [ ]:
import tensorflow as tf
import datetime
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import CSVLogger

# 模型
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
])

# 编译模型的 优化函数，损失函数，评估指标
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 日志目录
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# 学习率调度器:根据预定义的函数在每个 epoch 开始时动态设置学习率
def lr_schedule(epoch, lr):
    # 每 10 个 epoch，学习率减半
    if epoch % 10 == 0 and epoch != 0:
        return lr * 0.5
    return lr

# 在每个 epoch 结束后打印学习率
class PrintLearningRateCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        # 从优化器中获取当前学习率
        lr = self.model.optimizer.lr.numpy()
        print(f'\nEpoch {epoch+1}: Learning rate is {lr:.6f}')


# 定义回调函数
callbacks = [
    # 1. 保存验证集上表现最好的模型
    tf.keras.callbacks.ModelCheckpoint(
        filepath="models/best_model.h5",  # 保存路径
        monitor="val_loss",  # 监控的指标，如 'val_loss', 'accuracy'
        save_best_only=True, # 如果为 True，只保存在 monitor 上表现最好的模型
        mode='min',          # {'auto', 'min', 'max'}。对于 val_accuracy，要最大化所以是 'max'
        verbose=1            # 是否显示提示信息
    ),
    # 2. 早停：如果验证集 5 个 epoch 内没有改进，就停止训练
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",       # 监控验证损失
        patience=5,               # 容忍轮数。如果 5 个 epoch 后 val_loss 都没有改善，则停止
        restore_best_weights=True # 是否从停止的 epoch 中恢复模型为最佳权重。非常有用！
    ),
    # 3. 学习率调度器 OR ReduceLROnPlateau (建议二选一)
    # 选项A: 动态调整，验证集 loss 停滞时降低学习率
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss', 
        factor=0.5,        # 学习率减半 (new_lr = lr * factor)
        patience=3,        # 等待 3 个 epoch 无改善
        min_lr=1e-7,       # 学习率的下限
        verbose=1          # 打印学习率改变信息
    ),
    #  或选项B: 固定 schedule 
    # LearningRateScheduler(
    #     lr_schedule, 
    #     verbose=1
    # ), 
    # 学习率调度器和 ReduceLROnPlateau 的潜在冲突，同时使用可能会导致学习率变化过于激进，根据需求选择其中一种
    # 注意：如果使用学习率调度器，--优化器--的初始学习率可能需要明确设置
    
    # 4. 记录日志供 TensorBoard 可视化
    tf.keras.callbacks.TensorBoard(
        log_dir=log_dir,      # 日志目录
        histogram_freq=1,     # 每隔多少个 epoch 记录一次激活和权重的直方图
    ) ,
    
    # 5.在每个 epoch 结束后打印学习率,
    PrintLearningRateCallback()
    
    # 6.将每个 epoch 的损失和指标流式传输到 CSV 文件。
    CSVLogger('logs/training_log.csv')

]

# 训练
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          epochs=20,
          callbacks=callbacks)

## 4. 工作原理

回调在训练过程中会触发不同的 hook（钩子），例如：

- `on_epoch_begin` / `on_epoch_end`

- `on_batch_begin` / `on_batch_end`

- `on_train_begin` / `on_train_end`

你可以继承 `tf.keras.callbacks.Callback` 来实现 自定义回调。

---
## 总结
| 回调函数           | 核心功能             | 关键参数                          |
| ------------------ | -------------------- | --------------------------------- |
| `ModelCheckpoint`    | 保存模型             | `filepath`, `monitor`, `save_best_only` |
| `EarlyStopping`      | 提前终止训练         | `monitor`, `patience`, `restore_best_weights` |
| `ReduceLROnPlateau`  | 动态降低学习率       | `monitor`, `factor`, `patience`         |
| `TensorBoard`        | 可视化训练过程       | `log_dir`, `histogram_freq`           |
| `LearningRateScheduler` | 按计划调整学习率   | `schedule` (一个函数)               |
| `CSVLogger`          | 记录训练日志到文件   | `filename`                          |
